# Importación de librerias

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git
!pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tensorflow/examples.git to /tmp/pip-req-build-h8n3_3mc
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/examples.git /tmp/pip-req-build-h8n3_3mc
  Resolved https://github.com/tensorflow/examples.git to commit a71f7e38b0ce56e9f39d8049e836ea8f7af74f83
  Preparing metadata (setup.py) ... done
  Created wheel for tensorflow-examples: filename=tensorflow_examples-a71f7e38b0ce56e9f39d8049e836ea8f7af74f83_-py3-none-any.whl size=299718 sha256=e6ff4c3383a22f0c37cb16a82f29b95550ad3ae724e20a4fd5dfb976a5f4ad42
  Stored in directory: /tmp/pip-ephem-wheel-cache-qr6tyboe/wheels/4e/f5/c2/bfe75b834c9028b2529023bf74541c46ead531b513a8010d21
Failed to build tensorflow-examples
  Running setup.py install for tensorflow-examples ... done
  DEPRECATION: tensorflow-examples was installed using the legacy 'setup.py install' method, because a wheel c

In [ ]:
import tensorflow as tf
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as px

import PIL.Image

import cv2
from tensorflow_examples.models.pix2pix import pix2pix

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Creación de variables y fuciones

## Ajustes GAN

In [ ]:
def generator_gta():
    for path in images_GTA:
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (286,286))
        yield img


def generator_cityscapes():
    for path in images_Reales:
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (286,286))
        yield img


def preprocess_imgs(img):
    ## Convertimos las imágenes de int a float
    img = tf.cast(img, tf.float32)

    ## Normalizamos al rango [-1,1]
    img = (img/127.5) - 1

    ## Extraemos recortes aleatorios de 256x256
    img = tf.image.random_crop(img, size=[256, 256, 3])

    return img


def renormalize(img, contrast=1):
    return img*0.5*contrast + 0.5

In [ ]:
OUTPUT_CHANNELS = 3

generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)


generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

checkpoint_path = "./Checkpoints/"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

## Ajustes visualización

In [ ]:
def one_hot_mask(y):
    ''' Do the one hot encoding for the masks.
  
    Arguments:
        - y (tf tensor): Mask of shape (height, width, 3)

    Returns:
        - mask (tf tensor): Mask after do the one hot. Shape (height, width, num_classes) '''

    one_hot_map = []
    for color in colors:
        class_map = tf.reduce_all(tf.equal(y, color), axis = -1)
        one_hot_map.append(class_map)
    mask = tf.cast(tf.stack(one_hot_map, axis = -1), tf.int32)
    return mask


def load_image(folder, file, height = 96, width = 256, crop = False):
    ''' Load and preprocess a train image by:
        - Crop the image to not have the Mercedes-Benz star
        - Resize the image to (height, width)
        - Normalize the image to [0, 1]
  
    Arguments:
        - folder (string): Path to the folder
        - file (string): Name of the file to load
        - height (int): Height to resize -- 96
        - width (int): Width to resize -- 256
        - crop (bool): Crpo the image or not -- True

    Returns:
        - image (tf tensor): Preprocessed image '''

    # Load the image (png)
    image = tf.io.read_file(folder + '/' + file)
    image = tf.cast(tf.image.decode_png(image, channels = 3), tf.float32)

    # Crop the image
    if crop:
        image = tf.image.crop_to_bounding_box(image, 0, 0, 768, 2048)

    # Resize the image
    image = tf.image.resize(image, (height, width))

    # Normalize the image
    image = tf.cast(image, tf.float32)/255.0
    return image


def load_mask(folder, file, height = 96, width = 256, one_hot = True, crop = False):
    ''' Load and preprocess a train mask by:
        - Crop the image to not have the Mercedes-Benz star
        - Resize the image to (height, width)
        - Reshaping the mask from (height, width, 3) to (height, width, 30): One hot encoding
  
    Arguments:
        - folder (string): Path to the folder
        - file (string): Name of the file to load
        - height (int): Height to resize -- 96
        - width (int): Width to resize -- 256
        - one_hot (bool): Do one hot encoding or not -- True
        - crop (bool): Crpo the image or not -- True

    Returns:
        - image (tf tensor): Preprocessed mask '''

    # Load the mask (png)
    image = tf.io.read_file(folder + '/' + file)
    image = tf.cast(tf.image.decode_png(image, channels = 3), tf.int32)

    # Crop the mask
    if crop:
        image = tf.image.crop_to_bounding_box(image, 0, 0, 768, 2048)

    # Resize the mask
    image = tf.image.resize(image, (height, width), method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # One hot encoding
    if one_hot:
        image = one_hot_mask(image)
    return image


def one_hot_to_color_mask(mask, colors, img_height = 96, img_width = 256):
    ''' Convert from the mask from the classes with highest probablities to the correct color. From (96, 256, 1) to (96, 256, 3).
  
    Arguments:
        - mask (tf tensor): Mask with the classes with highest probabilities
        - colors (list): List with the class colors
        - img_height (int): Height of the images -- 96
        - img_width (int): Width of the images -- 256

    Returns:
        - color_mask (tf tensor): Color mask '''

    color_mask = np.zeros((img_height, img_width, channels)).astype('float')
    for c in range(len(colors)):
        color_true = mask == c
        for i in range(3):
            color_mask[:,:,i] += color_true*colors[c][i]

    color_mask = tf.cast(color_mask, dtype = tf.int32)

    return color_mask

def load_train(image_name, mask_name):
    ''' Load and preprocess a train image and its mask
  
    Arguments:
        - image_name (string): Name of the image to load
        - mask_name (string): Name of the mask to load

    Returns:
        - image (tf tensor): Preprocessed image
        - mask (tf tensor): Preprocessed mask '''

    image = load_image(train_images_folder_path, image_name, img_height, img_width)
    mask = load_mask(train_mask_folder_path, mask_name, img_height, img_width)
    return image, mask

In [ ]:
train_images_folder_path = "/content/drive/My Drive" 
train_mask_folder_path = "/content/drive/MyDrive/GTA Segmentacion" 

img_height, img_width, channels = 96, 256, 3

batch_size = 2

colors = np.array([(0, 0, 0), (111, 74, 0), (81, 0, 81), (128, 64, 128), (244, 35, 232), (250, 170, 160), (230, 150, 140), (70, 70, 70), (102, 102, 156), (190, 153, 153), (180, 165, 180), 
                   (150, 100, 100), (150, 120, 90), (153, 153, 153), (250, 170, 30), (220, 220, 0), (107, 142, 35), (152, 251, 152), (70, 130, 180), (220, 20, 60), (255, 0, 0), ( 0, 0, 142), 
                   ( 0, 0, 70), (0, 60, 100), (0, 0, 90), (0, 0, 110), (0, 80, 100), (0, 0, 230), (119, 11, 32), (0, 0, 142)], dtype = np.int32)

model = tf.keras.models.load_model('/content/drive/My Drive/Modelo_segmentacion_UNET/best_model_weights_and_architecture')

# rango = ["Original", "CycleGAN-6", "CycleGAN-10", "CycleGAN-12"]

# Inicialización

In [ ]:
path_result = "/content/drive/MyDrive/Resultados/"

path_GTA = "/content/drive/MyDrive/GTA/"
path_Reales = "/content/drive/MyDrive/Reales/"

itera = 60  # Número de imagenes de prueba

list_GTA = os.listdir(path_GTA)
images_GTA = random.choices(list_GTA, k=itera)

list_Reales = os.listdir(path_Reales)
images_Reales = random.choices(list_Reales, k=itera)

for i in range(itera):
  images_GTA[i] = path_GTA + images_GTA[i]
  images_Reales[i] = path_Reales + images_Reales[i]

## Cambiar alguna imagen si hace falta

In [ ]:
a = ["01526.png", "00031.png", "00513.png", "01166.png", "01151.png", "01782.png", "01550.png", "01450.png", "00557.png", "02078.png",  # 10 primeros
     "01575.png", "00370.png", "01401.png", "01999.png", "00203.png", "00560.png", "00142.png", "02074.png", "02406.png", "00467.png",
     "01889.png", "00466.png", "02307.png", "00529.png", "02071.png", "00045.png", "00301.png", "00491.png", "00078.png", "01289.png",
     "01862.png", "02264.png", "01297.png", "01385.png", "01348.png", "02019.png", "02099.png", "00242.png", "01101.png", "02313.png",
     "01655.png", "00923.png", "01495.png", "01947.png", "02404.png", "01293.png", "00733.png", "01579.png", "01701.png", "00248.png", 
     "01597.png", "01665.png", "01909.png", "00857.png", "00646.png", "02134.png", "00993.png", "01394.png", "01600.png", "01012.png"]

b = ["bremen_000152_000019_leftImg8bit.png", "stuttgart_000115_000019_leftImg8bit.png", "dusseldorf_000141_000019_leftImg8bit.png",
      "monchengladbach_000000_007098_leftImg8bit.png", "cologne_000006_000019_leftImg8bit.png", "bremen_000256_000019_leftImg8bit.png",
      "bremen_000190_000019_leftImg8bit.png", "cologne_000072_000019_leftImg8bit.png", "cologne_000064_000019_leftImg8bit.png", "strasbourg_000000_029020_leftImg8bit.png", # 10 primeros
      "aachen_000137_000019_leftImg8bit.png", "dusseldorf_000030_000019_leftImg8bit.png", "bremen_000004_000019_leftImg8bit.png", "bremen_000085_000019_leftImg8bit.png",
      "dusseldorf_000026_000019_leftImg8bit.png", "bochum_000000_009951_leftImg8bit.png", "tubingen_000120_000019_leftImg8bit.png", "hamburg_000000_032906_leftImg8bit.png",
      "hanover_000000_055800_leftImg8bit.png", "monchengladbach_000000_033454_leftImg8bit.png", "stuttgart_000065_000019_leftImg8bit.png", "bremen_000183_000019_leftImg8bit.png",
      "bremen_000079_000019_leftImg8bit.png", "stuttgart_000087_000019_leftImg8bit.png", "tubingen_000035_000019_leftImg8bit.png", "strasbourg_000001_022151_leftImg8bit.png",
      "aachen_000140_000019_leftImg8bit.png", "cologne_000094_000019_leftImg8bit.png", "jena_000052_000019_leftImg8bit.png", "tubingen_000059_000019_leftImg8bit.png",
      "aachen_000009_000019_leftImg8bit.png", "darmstadt_000074_000019_leftImg8bit.png", "strasbourg_000000_013863_leftImg8bit.png", "aachen_000097_000019_leftImg8bit.png",
      "dusseldorf_000033_000019_leftImg8bit.png", "cologne_000145_000019_leftImg8bit.png", "krefeld_000000_021222_leftImg8bit.png", "hamburg_000000_078407_leftImg8bit.png",
      "hamburg_000000_053486_leftImg8bit.png", "hanover_000000_026804_leftImg8bit.png", "bremen_000121_000019_leftImg8bit.png", "stuttgart_000012_000019_leftImg8bit.png",
      "bremen_000135_000019_leftImg8bit.png", "jena_000070_000019_leftImg8bit.png", "bochum_000000_023040_leftImg8bit.png", "jena_000108_000019_leftImg8bit.png",
      "stuttgart_000155_000019_leftImg8bit.png", "bochum_000000_030913_leftImg8bit.png", "strasbourg_000001_031272_leftImg8bit.png", "tubingen_000075_000019_leftImg8bit.png",
      "tubingen_000048_000019_leftImg8bit.png", "cologne_000005_000019_leftImg8bit.png", "dusseldorf_000097_000019_leftImg8bit.png", "bremen_000059_000019_leftImg8bit.png",
      "hamburg_000000_098400_leftImg8bit.png", "hamburg_000000_081299_leftImg8bit.png", "aachen_000004_000019_leftImg8bit.png", "hamburg_000000_079376_leftImg8bit.png",
      "bremen_000112_000019_leftImg8bit.png", "monchengladbach_000000_006518_leftImg8bit.png"]


for i in range(itera):
  images_GTA[i] = path_GTA + a[i]
  images_Reales[i] = path_Reales + b[i]

## Para saber que imágenes que se han escogido

In [ ]:
for name in images_GTA:
  print(name.split("/")[-1])
print("------------------------------------------------")
for name in images_Reales:
  print(name.split("/")[-1])

## Creamos la estructura de los datos

In [ ]:
dataset_gta = tf.data.Dataset.from_generator(
    generator_gta,
    output_signature=(
        tf.TensorSpec((286, 286, 3), tf.int32)
    )
)

dataset_cityscapes = tf.data.Dataset.from_generator(
    generator_cityscapes,
    output_signature=(
        tf.TensorSpec((286, 286, 3), tf.int32)
    )
)

dataset_gta_rdy = dataset_gta.map(preprocess_imgs)
dataset_cityscapes_rdy = dataset_cityscapes.map(preprocess_imgs)

dataset_full = tf.data.Dataset.zip((dataset_gta_rdy, dataset_cityscapes_rdy))

# Automático para muchos

In [ ]:
for num in range(0, itera):

# GAN  &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
  i = 0
  for a, b in dataset_full.batch(1):
    if i==num:
      img_gta = a
      img_cs = b
    i += 1

  ckpt.restore('/content/drive/MyDrive/CycleGTA/ckpt-6')
  Output_GAN_6 = generator_g.predict(img_gta)

  ckpt.restore('/content/drive/MyDrive/CycleGTA/ckpt-10')
  Output_GAN_10 = generator_g.predict(img_gta)

  ckpt.restore('/content/drive/MyDrive/CycleGTA/ckpt-12')
  Output_GAN_12 = generator_g.predict(img_gta)

# Guardamos los resultados
  fig, axes = plt.subplots(1,1)
  plt.imshow(renormalize(Output_GAN_6.squeeze())), plt.axis('off')
  plt.savefig(path_result + "Output_GAN_6_" + str(num+1) + ".png", bbox_inches="tight", pad_inches=0)

  fig, axes = plt.subplots(1,1)
  plt.imshow(renormalize(Output_GAN_10.squeeze())), plt.axis('off')
  plt.savefig(path_result + "Output_GAN_10_" + str(num+1) + ".png", bbox_inches="tight", pad_inches=0)

  fig, axes = plt.subplots(1,1)
  plt.imshow(renormalize(Output_GAN_12.squeeze())), plt.axis('off')
  plt.savefig(path_result + "Output_GAN_12_" + str(num+1) + ".png", bbox_inches="tight", pad_inches=0)


# Segmentación  &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
  images_name = ["GTA/" + images_GTA[num].split("/")[-1], "Resultados/Output_GAN_6_" + str(num+1) + ".png",
                 "Resultados/Output_GAN_10_" + str(num+1) + ".png", "Resultados/Output_GAN_12_" + str(num+1) + ".png"]

  mask_names = ((images_name[0].split("/")[-1] + " ") * 4).split()

  dataset = tf.data.Dataset.from_tensor_slices((images_name, mask_names))
  dataset = dataset.map(load_train, num_parallel_calls = tf.data.experimental.AUTOTUNE)
  dataset = dataset.batch(batch_size)

  predictions = model.predict(dataset, batch_size = 10)
  predictions = np.argmax(predictions, axis = 3)

  idx = range(len(images_name))
  visualize_images = [load_image(train_images_folder_path, images_name[i], img_height, img_width) for i in idx]
  visualize_masks = [load_mask(train_mask_folder_path, mask_names[i], img_height, img_width, one_hot = False) for i in idx]
  preds = predictions[idx]

  iou_score = []
  iou_score_half = []

  plt.figure(figsize = (16, 4))
  plt.subplots_adjust(hspace = 0.1)

  for k in range(4):
    plt.subplot(3, 4, k + 1)
    plt.imshow(visualize_images[k])
    # if k==0:
    #   plt.ylabel("Input")
    # plt.title(rango[k])
    plt.yticks(())
    plt.xticks(())

    plt.subplot(3, 4, k + 5)
    plt.imshow(visualize_masks[k])
    # if k==0:
    #   plt.ylabel("Ground Truth")
    plt.yticks(())
    plt.xticks(())

    output = one_hot_to_color_mask(preds[k], colors)
# IoU calculation
    intersection = 0
    union = img_height * img_width

    for i in range(img_height):
      for j in range(img_width):
        a = visualize_masks[k][i-1,j-1,:] == output[i-1,j-1,:]
        if sum(a.numpy())==3:
          intersection += 1

    iou_score.append(round(intersection/union, ndigits=2))
    
# Calcula de la mitad hacia abajo
    intersection = 0
    union = int(img_height/2) * img_width

    for i in range(int(img_height/2), img_height):
        for j in range(img_width):
          a = visualize_masks[k][i-1,j-1,:] == output[i-1,j-1,:]
          if sum(a.numpy())==3:
            intersection += 1

    iou_score_half.append(round(intersection/union, ndigits=2))
    
    plt.subplot(3, 4, k + 9)
    plt.imshow(output)
    # if k==0:
    #   plt.ylabel("Output")
    plt.xlabel("IoU: %s" % iou_score[k])
    plt.yticks(())
    plt.xticks(())

  plt.savefig(path_result + "Output_GTA_" + str(num+1) + ".png", bbox_inches="tight", pad_inches=0)

  print("Completado el", str(num+1))